In [1]:
import numpy as np
import pandas as pd
import datetime
import os

$$\frac{\partial V}{\partial t}+r\left( S \frac{\partial V}{\partial S}-V\right)+\frac{1}{2}\sigma^2[\frac{\partial^2 W(S,t)}{\partial S^2}]S^2\frac{\partial^2 W(S,t)}{\partial S^2}=0$$

$$ W(S,T)=F(S)$$

where $W^+$ is obtained setting
$$ \sigma [\frac{\partial^2 W}{\partial S^2}]=\begin{cases} \sigma_{\max} & if \frac{\partial^2 W}{\partial S^2}\geq 0 \\ \sigma_{\min} & if \frac{\partial^2 W}{\partial S^2}< 0 \end{cases}$$

and $W^-$ is obtained setting
$$ \sigma [\frac{\partial^2 W}{\partial S^2}]=\begin{cases} \sigma_{\max} & if \frac{\partial^2 W}{\partial S^2}\leq 0 \\ \sigma_{\min} & if \frac{\partial^2 W}{\partial S^2}> 0 \end{cases}$$

In [17]:
for file in os.listdir('../option_data/spy_data'):
    if file[-4:] == '.csv':
        print(file)
        df = pd.read_csv('../option_data/spy_data/' + file)        
        
        # moving to datetime and making features
        df['quote_datetime'] = pd.to_datetime(df['quote_datetime'])
        df['expiration'] = pd.to_datetime(df['expiration'])
        df['quote_date'] = df['quote_datetime'][0].date()
        df['quote_date'] = pd.to_datetime(df['quote_date'])
        
        # getting only 4:00 quotes
        eod = datetime.datetime.combine(df['quote_datetime'][0].date(), datetime.time(16,0, 0))
        df = df.loc[df['quote_datetime'] == eod]
        
        # getting time to expiration and moneyness
        df['T'] = df['expiration'] - df['quote_date']
        df['T'] = df['T'].dt.days
        df['moneyness'] = df['active_underlying_price'] / df['strike'] 

        
        # filtering for research paper criteria
        df = df.loc[(df['close']!=0) & (df['implied_volatility']!=0) & (df['T']>=20) & (df['T']<=365) & (df['moneyness']>0.7) & (df['moneyness']<1.3)]

        calls = df.loc[df['option_type']=='C'][['T', 'moneyness', 'implied_volatility', 'active_underlying_price', 'strike']]
        puts = df.loc[df['option_type']=='P'][['T', 'moneyness', 'implied_volatility', 'active_underlying_price', 'strike']]
        break

UnderlyingOptionsIntervals_23400sec_calcs_oi_2023-10-09.csv


In [15]:
calls

,T,moneyness,implied_volatility,active_underlying_price,strike
5476,25,1.290373,0.3985,432.275,335.0
5480,25,1.271397,0.3666,432.275,340.0
5504,25,1.168311,0.2941,432.275,370.0
5508,25,1.152733,0.2819,432.275,375.0
5512,25,1.137566,0.2662,432.275,380.0
...,...,...,...,...,...
12824,357,0.891289,0.1499,432.275,485.0
12828,357,0.882194,0.1422,432.275,490.0
12832,357,0.873283,0.1393,432.275,495.0
12836,357,0.864550,0.1363,432.275,500.0


In [16]:
puts

,T,moneyness,implied_volatility,active_underlying_price,strike
5478,25,1.290373,0.4147,432.275,335.0
5482,25,1.271397,0.3975,432.275,340.0
5486,25,1.252971,0.3833,432.275,345.0
5490,25,1.235071,0.3654,432.275,350.0
5494,25,1.217676,0.3497,432.275,355.0
...,...,...,...,...,...
12662,357,0.993736,0.1812,432.275,435.0
12818,357,0.910053,0.1519,432.275,475.0
12822,357,0.900573,0.1465,432.275,480.0
12826,357,0.891289,0.1403,432.275,485.0
